# M2 Example using gridded data

In [5]:
import pandas as pd
import numpy as np
import xarray as xr

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import YearLocator, WeekdayLocator, MonthLocator, DayLocator, HourLocator, DateFormatter

import cmocean
#print(cmocean.cm.cmapnames)

In [6]:
from erddapy.erddapy import ERDDAP

In [7]:
server_url='http://akutan.pmel.noaa.gov:8080/erddap'
dataset = '1hr_gridded_datasets_Mooring_18bsm2a_final'

e = ERDDAP(server=server_url)

e.constraints = None
e.protocol = 'griddap'

opendap_url = e.get_download_url(
    dataset_id=dataset,
    response='opendap',
)

print(opendap_url)

http://akutan.pmel.noaa.gov:8080/erddap/griddap/1hr_gridded_datasets_Mooring_18bsm2a_final


In [8]:
from netCDF4 import Dataset

with Dataset(opendap_url) as nc:
    print(nc.summary)

OSError: [Errno -68] NetCDF: I/O failure: b'http://akutan.pmel.noaa.gov:8080/erddap/griddap/1hr_gridded_datasets_Mooring_18bsm2a_final'

In [ ]:
e.dataset_id=dataset
e.response = 'nc'
ds = e.to_xarray(decode_times=True)

In [ ]:
### specify primary bulk figure parameters
fontsize = 20
labelsize = 16
#plotstyle = 'seaborn'
max_xticks = 10
plt.style.use('seaborn-ticks')
mpl.rcParams['svg.fonttype'] = 'none'
mpl.rcParams['ps.fonttype'] = 42 #truetype/type2 fonts instead of type3
mpl.rcParams['pdf.fonttype'] = 42 #truetype/type2 fonts instead of type3
mpl.rcParams['axes.grid'] = False
mpl.rcParams['axes.edgecolor'] = 'black'
mpl.rcParams['axes.linewidth'] = 1.5
mpl.rcParams['axes.labelcolor'] = 'black'
mpl.rcParams['grid.linestyle'] = '--'
mpl.rcParams['grid.linestyle'] = '--'
mpl.rcParams['xtick.major.size'] = 4
mpl.rcParams['xtick.minor.size'] = 1
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['xtick.minor.width'] = 1
mpl.rcParams['ytick.major.size'] = 4
mpl.rcParams['ytick.minor.size'] = 1
mpl.rcParams['ytick.major.width'] = 2
mpl.rcParams['ytick.minor.width'] = 1
mpl.rcParams['ytick.direction'] = 'out'
mpl.rcParams['xtick.direction'] = 'out'
mpl.rcParams['ytick.color'] = 'black'
mpl.rcParams['xtick.color'] = 'black'

In [ ]:

fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=4, sharex=True, figsize=(8.5,5.5))

ax1.pcolormesh(ds.time,ds.depth,ds.temperature,
                     cmap=cmocean.cm.thermal)
ax2.pcolormesh(ds.time,ds.depth,ds.temperature.diff(dim='depth'),
                     cmap=cmocean.cm.balance)
ax3.pcolormesh(ds.time,ds.depth,ds.salinity,
                     cmap=cmocean.cm.haline)
ax4.pcolormesh(ds.time,ds.depth,ds.Chlorophyll_Fluorescence,
                     cmap=cmocean.cm.algae)


ax1.invert_yaxis()
ax2.invert_yaxis()
ax3.invert_yaxis()
ax4.invert_yaxis()

xfmt = mdates.DateFormatter('%d-%b\n%y')
ax4.xaxis.set_major_formatter(xfmt)
ax4.xaxis.set_major_locator(MonthLocator())
ax4.xaxis.set_minor_locator(DayLocator())


In [ ]:
import holoviews as hv
from holoviews import opts

hv.extension('bokeh')

hv_ds = hv.Dataset(ds['temperature'])[:,:]

data = hv_ds.to(hv.QuadMesh,kdims=["time", "depth"]).options(**dict(invert_yaxis=True))
data

In [ ]:
ds.longitude.max()

In [ ]:
### interp test

newdepth = np.linspace(0, 72,73)

filled = ds.interpolate_na(dim='depth',limit=1)
dsi = filled.interp(depth=newdepth)

fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=4, sharex=True, figsize=(8.5,5.5))

ax1.pcolormesh(dsi.time,dsi.depth,dsi.temperature,
                     cmap=cmocean.cm.thermal)
ax2.pcolormesh(dsi.time,dsi.depth,dsi.temperature.diff(dim='depth'),
                     cmap=cmocean.cm.balance)
ax3.pcolormesh(dsi.time,dsi.depth,dsi.salinity,
                     cmap=cmocean.cm.haline)
ax4.pcolormesh(dsi.time,dsi.depth,dsi.Chlorophyll_Fluorescence,
                     cmap=cmocean.cm.algae)


ax1.invert_yaxis()
ax2.invert_yaxis()
ax3.invert_yaxis()
ax4.invert_yaxis()

xfmt = mdates.DateFormatter('%d-%b\n%y')
ax4.xaxis.set_major_formatter(xfmt)
ax4.xaxis.set_major_locator(MonthLocator())
ax4.xaxis.set_minor_locator(DayLocator())


In [ ]:
### loop over all M2 datasets

e = ERDDAP(server=server_url)
df = pd.read_csv(e.get_search_url(response='csv', search_for='1hr_'))
print(df['Dataset ID'].values)

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=4, sharex=True, figsize=(8.5,5.5))

for dataset_id in df['Dataset ID'].values:
    if ('m2a' in dataset_id) or ('bs2c' in dataset_id):
        print(dataset_id)
        
        e.protocol = 'griddap'
        e.dataset_id=dataset_id
        e.response = 'nc'
        ds = e.to_xarray(decode_times=True)
        
        newdepth = np.linspace(0, 72,73)

        try:
            filled = ds.interpolate_na(dim='depth',limit=2)
            dsi = filled.interp(depth=newdepth)

            ax1.pcolormesh(dsi.time,dsi.depth,dsi.temperature,
                           vmin=-2,vmax=14,
                           cmap=cmocean.cm.thermal)
            ax2.pcolormesh(dsi.time,dsi.depth,dsi.temperature.diff(dim='depth'),
                           vmin=-5,vmax=5,
                           cmap=cmocean.cm.balance)
            ax3.pcolormesh(dsi.time,dsi.depth,dsi.salinity,
                           vmin=31,vmax=32,
                           cmap=cmocean.cm.haline)
            ax4.pcolormesh(dsi.time,dsi.depth,dsi.Chlorophyll_Fluorescence,
                           vmin=0,vmax=20,
                           cmap=cmocean.cm.algae)
        except:
            continue

ax1.invert_yaxis()
ax2.invert_yaxis()
ax3.invert_yaxis()
ax4.invert_yaxis()

xfmt = mdates.DateFormatter('%d-%b\n%y')
ax4.xaxis.set_major_formatter(xfmt)
ax4.xaxis.set_major_locator(YearLocator())
ax4.xaxis.set_minor_locator(MonthLocator())



In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=4, sharex=True, figsize=(8.5,5.5))

for dataset_id in df['Dataset ID'].values:
    if ('m2a' in dataset_id) or ('bs2c' in dataset_id):
        print(dataset_id)
        
        e.protocol = 'griddap'
        e.dataset_id=dataset_id
        e.response = 'nc'
        ds = e.to_xarray(decode_times=True)
        

        ax1.pcolormesh(ds.time,ds.depth,ds.temperature,
                       vmin=-2,vmax=14,
                       cmap=cmocean.cm.thermal)
        ax2.pcolormesh(ds.time,ds.depth,ds.temperature.diff(dim='depth'),
                       vmin=-5,vmax=5,
                       cmap=cmocean.cm.balance)
        ax3.pcolormesh(ds.time,ds.depth,ds.salinity,
                       vmin=31,vmax=32,
                       cmap=cmocean.cm.haline)
        ax4.pcolormesh(ds.time,ds.depth,ds.Chlorophyll_Fluorescence,
                       vmin=0,vmax=20,
                       cmap=cmocean.cm.algae)


ax1.invert_yaxis()
ax2.invert_yaxis()
ax3.invert_yaxis()
ax4.invert_yaxis()

xfmt = mdates.DateFormatter('%d-%b\n%y')
ax4.xaxis.set_major_formatter(xfmt)
ax4.xaxis.set_major_locator(YearLocator())
ax4.xaxis.set_minor_locator(MonthLocator())

